In [7]:
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv('/Users/marie-sophiechenevier/Dropbox/8-Jedha/GitHub/2-Jedha_Fullstack/4_data_collection_management/4_ETL_processes/credentials.env')

# Récupérer les informations de connexion
HOSTNAME = os.getenv("HOSTNAME")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
DBNAME = os.getenv("DBNAME")

# Créer une connexion à la base de données MySQL
engine = create_engine(f"mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True)

# Exécuter une requête
with engine.connect() as conn:
    res = conn.execute(text("SELECT * FROM food_truck;"))
    data = res.fetchall()
    print(data)


ProgrammingError: (mysql.connector.errors.ProgrammingError) 1049 (42000): Unknown database 'msintechtwo-firstdb-terminal'
(Background on this error at: https://sqlalche.me/e/20/f405)